In [ ]:
# Install necessary packages (if running in Colab)
!pip install opencv-python
from google.colab.patches import cv2_imshow  # Only needed if you're using Colab
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import numpy as np

def face_mask_detector(frame):
    # Create a blurred version of the background
    blurred_frame = cv2.GaussianBlur(frame, (21, 21), 0)

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)

    # Create a mask with the same shape and type as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract face region
        face_frame = frame[y:y+h, x:x+w]

        # Preprocess face for mask model
        face_frame_rgb = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame_resized = cv2.resize(face_frame_rgb, (224, 224))
        face_frame_array = img_to_array(face_frame_resized)
        face_frame_array = np.expand_dims(face_frame_array, axis=0)
        face_frame_array = preprocess_input(face_frame_array)

        # Predict mask or no mask
        faces_list = [face_frame_array]
        preds = model.predict(faces_list)

        # Process prediction results
        for pred in preds:
            (mask_prob, withoutMask) = pred
            label = "Mask" if mask_prob > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            label = "{}: {:.2f}%".format(label, max(mask_prob, withoutMask) * 100)

            # Draw label and bounding box on original frame
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            # Keep the face region in focus by copying it onto the mask
            mask[y:y+h, x:x+w] = face_frame  # Assign the face area to the mask

    # Combine blurred background and the mask to keep the face in focus
    output_frame = np.where(mask > 0, mask, blurred_frame)

    return output_frame

# Video processing function
def process_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Process each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no more frames are available

        # Detect face masks in the current frame
        output_frame = face_mask_detector(frame)

        # Display the frame (in Colab use cv2_imshow)
        cv2_imshow(output_frame)  # For Colab
        # cv2.imshow("Face Mask Detection", output_frame)  # Uncomment for local execution

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close display window
    cap.release()
    cv2.destroyAllWindows()

# Call the function with your video path
process_video("video.mp4")
